In [6]:
import carla, time, pygame, cv2
import numpy as np

In [7]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()
carla_settings = world.get_settings()

In [8]:
def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def spawn_pedestrian(spawn_index=0):
    blueprint_library = world.get_blueprint_library()
    pedestrian_bp = blueprint_library.filter('walker.pedestrian.*')[0]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    pedestrian = world.spawn_actor(pedestrian_bp, spawn_point)
    return pedestrian

def callback(image):
    image.convert(carla.ColorConverter.CityScapesPalette)
    time.sleep(3)
    image.save_to_disk('output/%.6d.png' % image.frame)

def move_spectator_to(transform, spectator, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

In [14]:
vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle)
vehicle.set_autopilot(True)

spectator_transform = vehicle.get_transform()
move_spectator_to(spectator_transform, spectator)

camera.listen(callback)

for i in range(10):
    pedestrian = spawn_pedestrian(spawn_index=i)

AttributeError: 'Walker' object has no attribute 'set_autopilot'

In [15]:
if vehicle:
    vehicle.destroy()
if camera:
    camera.stop()
    camera.destroy()
for actor in world.get_actors().filter('walker.pedestrian.*'):
    actor.destroy()

: 